# Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

    если источник traffic_source равен yandex или google, то в source_type ставится organic
    для источников paid и email из России - ставим ad
    для источников paid и email не из России - ставим other
    все остальные варианты берем из traffic_source без изменений

In [360]:
%%time
import pandas as pd
 
df1 = pd.read_csv('visit_log.csv', sep = ';')
df1.loc[(df1.traffic_source.str.contains('paid|email') & df1.region.str.contains('Russia')),'source_type'] = 'ad'
df1.loc[(df1.traffic_source.str.contains('paid|email') & ~df1.region.str.contains('Russia')),'source_type'] = 'other'
df1.loc[~df1.traffic_source.str.contains('paid|email'),'source_type'] = df1.loc[~df1.traffic_source.str.contains('paid|email'),'traffic_source']
df1.loc[df1.traffic_source.str.contains('yandex|google'),'source_type'] = 'organic'
df1

Wall time: 124 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad


# Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

    Прочитайте содержимое файла с датафрейм
    Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном


In [417]:
df1 = pd.read_csv('URLs.txt',sep = '\t')
patt = r'/\d{8}-'
df1.loc[df1.url.str.contains(patt)]



Wall time: 3.74 ms


,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


# Задание 3

    Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [518]:
# Вычисляем среднее время жизни пользователя в часах
def life_user(u):
    return (u.max() - u.min())/3600

df1 = pd.read_csv('ratings.csv')
df1 = df1.groupby(['userId'],as_index = False).agg({'timestamp':life_user,'movieId':'count'})
df1.loc[df1['movieId'] > 100].sort_values('timestamp').reset_index()


,index,userId,timestamp,movieId
0,535,536,0.057222,109
1,608,609,0.235556,140
2,315,316,0.254444,140
3,216,217,0.272500,104
4,109,110,0.318889,120
...,...,...,...,...
253,623,624,127081.251944,1735
254,14,15,130942.637778,1700
255,426,427,132380.348889,294
256,546,547,139391.815278,2391


# Задание 4

    Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
    Необходимо сформировать две таблицы:

    таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
    аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [539]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
joined_1 = rzd.merge(auto,how = 'outer').merge(air,how = 'outer')
joined_2 = joined_1.merge(client_base, how = 'outer')
joined_2


,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
